In [ ]:
import numpy as np 
import numpy.random as npr

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.decomposition import PCA
import statsmodels.api as sm
from numpy.linalg import cond

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
N=2000
D=5 # number of features
mean = np.zeros(D)
corr = 0.9

In [ ]:
y_noise = 0.1
# designate the core feature
num_corefea = np.int(D/2)
true_cause = np.arange(num_corefea).astype(int)

## generate simulated datasets with core and spurious features
The outcome model is the same in training and testing; the outcome only depends on the core feature. 

In the training set, the covariates have high correlation. In the test set, the covariates have low correlation.

In [ ]:
# simulate strongly correlated features for training
train_cov = np.ones((D, D)) * corr + np.eye(D) * (1 - corr)
train_x_true = npr.multivariate_normal(mean, train_cov, size=N)
train_x_true = train_x_true * np.concatenate([-1 * np.ones(D//2), np.ones(D - D//2)])  # create both positive and negatively correlated covariates
# train_x_true = np.exp(npr.multivariate_normal(mean, train_cov, size=N)) # exponential of gaussian; no need to be gaussian

In [ ]:
# simulate weakly correlated features for testing
test_cov = np.ones((D, D)) * (1 - corr) + np.eye(D) * corr
test_x_true = npr.multivariate_normal(mean, test_cov, size=N)
# test_x_true = np.exp(npr.multivariate_normal(mean, test_cov, size=N))  # exponential of gaussian; no need to be gaussian

In [ ]:
# add observation noise to the x
# spurious correlation more often occurs when the signal to noise ratio is lower
x_noise = np.array(list(np.ones(num_corefea)*0.4) + list(np.ones(D-num_corefea)*0.3))

train_x = train_x_true + x_noise * npr.normal(size=[N,D])
test_x = test_x_true + x_noise * npr.normal(size=[N,D])

In [ ]:
print("\ntrain X correlation\n", np.corrcoef(train_x.T))
print("\ntest X correlation\n",np.corrcoef(test_x.T))


train X correlation
 [[ 1.          0.77539078 -0.79854585 -0.80515214 -0.80946749]
 [ 0.77539078  1.         -0.78604935 -0.79897645 -0.78826129]
 [-0.79854585 -0.78604935  1.          0.82249577  0.82893882]
 [-0.80515214 -0.79897645  0.82249577  1.          0.82461495]
 [-0.80946749 -0.78826129  0.82893882  0.82461495  1.        ]]

test X correlation
 [[1.         0.04767672 0.08736198 0.08612526 0.06286724]
 [0.04767672 1.         0.10137518 0.08131194 0.12368218]
 [0.08736198 0.10137518 1.         0.09296564 0.11387599]
 [0.08612526 0.08131194 0.09296564 1.         0.08053406]
 [0.06286724 0.12368218 0.11387599 0.08053406 1.        ]]


In [ ]:
# generate outcome
# toy model y = x + noise
truecoeff = npr.uniform(size=num_corefea) * 10
train_y = train_x_true[:,true_cause].dot(truecoeff) + y_noise * npr.normal(size=N)
test_y = test_x_true[:,true_cause].dot(truecoeff) + y_noise * npr.normal(size=N)

# baseline naive regression on all features

In [ ]:
# regularization parameter for ridge regression
alpha = 10

In [ ]:
def fitcoef(cov_train, train_y, cov_test=None, test_y=None):
	# linearReg
	print("linearReg")
	reg = LinearRegression()
	reg.fit(cov_train, train_y)
	print("coef", reg.coef_, "intercept", reg.intercept_)
	print("train accuracy", reg.score(cov_train, train_y))
	if cov_test is not None:
		print("test accuracy", reg.score(cov_test, test_y))

	# # linearReg with statsmodels
	# print("linearReg with statsmodels")
	# model = sm.OLS(train_y,sm.add_constant(cov_train, prepend=False))
	# result = model.fit()
	# print(result.summary())

	# ridgeReg
	print("ridgeReg")
	reg = Ridge(alpha=alpha)
	reg.fit(cov_train, train_y)
	print("coef", reg.coef_, "intercept", reg.intercept_)
	print("train accuracy", reg.score(cov_train, train_y))
	if cov_test is not None:
		print("test accuracy", reg.score(cov_test, test_y))

all three features have coefficient different from zeuo

test accuracy degrades much from training accuracy.

In [ ]:
print("\n###########################\nall features")

cov_train = np.column_stack([train_x])
cov_test = np.column_stack([test_x])

fitcoef(cov_train, train_y, cov_test, test_y)


###########################
all features
linearReg
coef [ 3.8260484   4.12551171 -1.73184093 -2.1531013  -1.94930994] intercept 0.06787967518858667
train accuracy 0.9507623665207684
test accuracy 0.5396228741066758
ridgeReg
coef [ 3.80026212  4.09681077 -1.74901472 -2.16332991 -1.96259499] intercept 0.06734786407779533
train accuracy 0.9507587255663613
test accuracy 0.533815209576548


next consider oracle, regression only on the core feature

In [ ]:
print("\n###########################\nall features")

cov_train = np.column_stack([train_x[:,true_cause]])
cov_test = np.column_stack([test_x[:,true_cause]])

fitcoef(cov_train, train_y, cov_test, test_y)


###########################
all features
linearReg
coef [6.52558241 6.48789148] intercept 0.07958701260146375
train accuracy 0.918613866755308
test accuracy 0.8532555333794934
ridgeReg
coef [6.50908351 6.47266454] intercept 0.07895626923886387
train accuracy 0.9186084068200724
test accuracy 0.8535213249364966


## causal-rep
now try adjust for pca factor, then learn feature coefficient, construct a prediction function using the learned feature mapping, predict on the test set

In [ ]:
# fit pca to high correlated training dataset
pca = PCA(n_components=1)
pca.fit(train_x)
pca.transform(train_x)

array([[ 0.1871552 ],
       [ 2.1947628 ],
       [-1.17747367],
       ...,
       [ 0.59919576],
       [-1.74839058],
       [-1.69757482]])

In [ ]:
# consider features 0,1 (have to consider a subset of features; 
# alternatively one can consider features 0,2
# cannot consider all three due to colinearity issues 
# (a.k.a. violation of overlap))
print("\n###########################\ncore + spurious 1 + pca")
candidate_trainfea = train_x[:,:-1]
candidate_testfea = test_x[:,:-1]
adjust_trainC = pca.transform(train_x)
cov_train = np.column_stack([candidate_trainfea, adjust_trainC])
print("linearReg")
feareg = LinearRegression()
feareg.fit(cov_train, train_y)
print("coef", feareg.coef_, "intercept", feareg.intercept_)
print("train accuracy", feareg.score(cov_train, train_y))


###########################
core + spurious 1 + pca
linearReg
coef [ 1.82556191  2.13568875  0.20943855 -0.180255    4.4070216 ] intercept -0.09848025696455924
train accuracy 0.9507623665207684


In [ ]:
# cond(candidate_trainfea.dot(candidate_trainfea.T))

above, after adjusting for pca factor, the spurious feature 1 returns close to zero coefficient

In [ ]:
# construct a prediction model using the learned 
# feature combination of "core + spurious 1"
learned_fea_train = candidate_trainfea.dot(feareg.coef_[:candidate_trainfea.shape[1]])[:,np.newaxis]
predreg = LinearRegression()
predreg.fit(learned_fea_train, train_y)
print("trainfea_coef", predreg.coef_, "intercept", predreg.intercept_)
print("trainfea accuracy", predreg.score(learned_fea_train, train_y))

trainfea_coef [3.29503926] intercept 0.08824633019287884
trainfea accuracy 0.9161251249691037


In [ ]:
# apply the prediction model on the test data
learned_fea_test = candidate_testfea.dot(feareg.coef_[:candidate_trainfea.shape[1]])[:,np.newaxis]
print("testfea accuracy", predreg.score(learned_fea_test, test_y))

testfea accuracy 0.8514300644432049


above, the test accuracy no longer degrades much from the training accuracy.

also note that the test accuracy is very close to the oracle accuracy.